In [ ]:
from pathlib import Path

from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from mlip_arena.models import REGISTRY, MLIPEnum
from mlip_arena.tasks.md import run as MD
from mlip_arena.tasks.utils import get_calculator
from prefect import flow
from prefect_dask import DaskTaskRunner

from ase import Atoms, units
from ase.build import molecule
from ase.io import read, write
from pymatgen.core import Molecule
from pymatgen.io.packmol import PackmolBoxGen

## Intial configuration

In [ ]:
h2 = molecule("H2")
o2 = molecule("O2")
h2o = molecule("H2O")

write("h2.xyz", h2)
write("o2.xyz", o2)
write("h2o.xyz", h2o)

In [ ]:
h2 = Molecule.from_file("h2.xyz")
o2 = Molecule.from_file("o2.xyz")
h2o = Molecule.from_file("h2o.xyz")

In [ ]:
molecules = []

for m, number in zip([h2, o2], [128, 64]):
    molecules.append(
        {
            "name": m.composition.to_pretty_string(),
            "number": number,
            "coords": m,
        }
    )

In [ ]:
tolerance = 2.0
input_gen = PackmolBoxGen(
    tolerance=tolerance,
    seed=1,
)
margin = 0.5 * tolerance

a = 30

packmol_set = input_gen.get_input_set(
    molecules=molecules,
    box=[margin, margin, margin, a - margin, a - margin, a - margin],
)
packmol_set.write_input(".")
packmol_set.run(".")

atoms = read("packmol_out.xyz")
atoms.cell = [a, a, a]
atoms.pbc = True

print(atoms)

write(f"{atoms.get_chemical_formula()}.extxyz", atoms)

## Run workflow

In [ ]:
atoms = read("H256O128.extxyz")
print(atoms)

In [ ]:
nodes_per_alloc = 1
gpus_per_alloc = 4
ntasks = 1

cluster_kwargs = dict(
    cores=1,
    memory="64 GB",
    shebang="#!/bin/bash",
    account="m4282",
    walltime="00:30:00",
    job_mem="0",
    job_script_prologue=[
        "source ~/.bashrc",
        "module load python",
        "source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena",
    ],
    job_directives_skip=["-n", "--cpus-per-task", "-J"],
    job_extra_directives=[
        "-J combustion-water",
        "-q debug",
        f"-N {nodes_per_alloc}",
        "-C gpu",
        f"-G {gpus_per_alloc}",
        "--exclusive",
    ],
    death_timeout=86400,
)

cluster = SLURMCluster(**cluster_kwargs)


print(cluster.job_script())
cluster.adapt(minimum_jobs=1, maximum_jobs=1)
client = Client(cluster)

In [ ]:
@flow(task_runner=DaskTaskRunner(address=client.scheduler.address), log_prints=True)
def combustion(atoms: Atoms):
    futures = []

    for model in MLIPEnum:
        if model.name != "MatterSim":
            continue

        future = MD.submit(
            atoms=atoms,
            calculator=get_calculator(
                calculator_name=model,
                calculator_kwargs=None,
            ),
            ensemble="nvt",
            dynamics="nose-hoover",
            time_step=None,
            dynamics_kwargs=dict(ttime=25 * units.fs, pfactor=None),
            total_time=1000_000,
            temperature=[300, 3000, 3000, 300],
            pressure=None,
            velocity_seed=0,
            traj_file=Path(REGISTRY[model.name]["family"])
            / f"{model.name}_{atoms.get_chemical_formula()}.traj",
            traj_interval=1000,
            restart=True,
        )

        futures.append(future)

    return [future.result() for future in futures]

In [ ]:
results = combustion(atoms)

In [ ]:
def combustion(atoms: Atoms):
    futures = []

    for model in MLIPEnum:
        if model.name != "MatterSim":
            continue

        future = MD(
            atoms=atoms,
            calculator=get_calculator(
                calculator_name=model,
                calculator_kwargs=None,
            ),
            ensemble="nvt",
            dynamics="nose-hoover",
            time_step=None,
            dynamics_kwargs=dict(ttime=25 * units.fs, pfactor=None),
            total_time=1000_000,
            temperature=[300, 3000, 3000, 300],
            pressure=None,
            velocity_seed=0,
            traj_file=Path(REGISTRY[model.name]["family"])
            / f"{model.name}_{atoms.get_chemical_formula()}.traj",
            traj_interval=1000,
            restart=True,
        )

        futures.append(future)

    return [future.result() for future in futures]


results = combustion(atoms)